
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning">
</div>


# Shuffle

Shuffle is a Spark mechanism that redistributes data so that it's grouped differently across partitions. This typically involves copying data across executors and machines and, while it's sometimes necessary, it can be a complex and costly operation.

In this demo we will see shuffle in action. Run the next cell to set up the lesson.

## REQUIRED - SELECT CLASSIC COMPUTE

Before executing cells in this notebook, please select your classic compute cluster in the lab. Be aware that **Serverless** is enabled by default. If you use Serverless, errors will be returned when setting compute runtime properties.

Follow these steps to select the classic compute cluster:

1. Navigate to the top-right of this notebook and click the drop-down menu to select your cluster. By default, the notebook will use **Serverless**.

1. If your cluster is available, select it and continue to the next cell. If the cluster is not shown:

  - In the drop-down, select **More**.

  - In the **Attach to an existing compute resource** pop-up, select the first drop-down. You will see a unique cluster name in that drop-down. Please select that cluster.

**NOTE:** If your cluster has terminated, you might need to restart it in order to select it. To do this:

1. Right-click on **Compute** in the left navigation pane and select *Open in new tab*.

1. Find the triangle icon to the right of your compute cluster name and click it.

1. Wait a few minutes for the cluster to start.

1. Once the cluster is running, complete the steps above to select your cluster.

## A. Classroom Setup

Run the following cell to configure your working environment for this course. It will also set your default catalog to your unique **labuser** catalog, and the default schema to **default**. All tables will be read from and written to this location.
<br></br>

**NOTE:** The `DA` object is only used in Databricks Academy courses and is not available outside of these courses. It will dynamically reference the information needed to run the course.

In [0]:
%run ./Includes/Classroom-Setup-3

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


Set the spark.sql.autoBroadcastJoinThreshold and spark.databricks.adaptive.autoBroadcastJoinThreshold configs to the defaults.


Course Catalog:,
Your Schema:,


Let's Check the Current Catalog and Schema

In [0]:
%sql
SELECT current_catalog(), current_schema()

current_catalog(),current_schema()
labuser10305556_1747289640,default


## B. Disable Caching

Run the following cell to set a Spark configuration variable that disables disk caching.

Turning disk caching off prevents Databricks from storing cloud storage files after the first query. This makes the effect of the optimizations more apparent by ensuring that files are always pulled from cloud storage for each query.

For more information, see [Optimize performance with caching on Databricks](https://docs.databricks.com/en/optimizations/disk-cache.html#optimize-performance-with-caching-on-databricks).

**NOTE:** This will not work in Serverless. Please use classic compute to turn off caching. If you're using Serverless, an error will be returned.

In [0]:
spark.conf.set('spark.databricks.io.cache.enabled', False)


## C. Creating Tables
Create the tables the following tables for the demonstration:
- **transactions**
- **stores**
- **countries**

### C1. Create the transactions table
Let's generate the data we will use in this demo. First, we'll synthesize data representing a set of sales transactions and write the data to a table named **transactions**.

The table will contain *150,000,000* rows.


In [0]:
from pyspark.sql.functions import *

## Drop the table if it exists
spark.sql('DROP TABLE IF EXISTS transactions')

## Create the Spark DataFrame and save as a table
(spark
 .range(0, 150000000, 1, 32)
 .select(
    'id',
    round(rand() * 10000, 2).alias('amount'),
    (col('id') % 10).alias('country_id'),
    (col('id') % 100).alias('store_id')
 )
 .write
 .mode('overwrite')
 .saveAsTable('transactions')
)

## Preview the table
display(spark.sql('SELECT * FROM transactions LIMIT 10'))

id,amount,country_id,store_id
75000000,9454.41,0,0
75000001,7661.94,1,1
75000002,6024.84,2,2
75000003,2700.64,3,3
75000004,4871.13,4,4
75000005,6125.23,5,5
75000006,1435.96,6,6
75000007,766.07,7,7
75000008,1180.76,8,8
75000009,3592.82,9,9



### C2. Create the stores table
Now let's synthesize data and write it to a table describing points of sale information named **stores**.

The table will contain 99 rows.

In [0]:
## Drop the table if it exists
spark.sql('DROP TABLE IF EXISTS stores')

## Create the Spark DataFrame
(spark
 .range(0, 99)
 .select(
    'id',
    round(rand() * 100, 0).alias('employees'),
    (col('id') % 10).alias('country_id'),
    expr('uuid()').alias('name')
    )
 .write
 .mode('overwrite')
 .saveAsTable('stores')
)

## Display the table
display(spark.sql('SELECT * FROM stores LIMIT 10'))

id,employees,country_id,name
24,72.0,4,6b4b90c4-b333-4be2-8586-6501ccb0d6bf
25,59.0,5,97cf2fdd-154f-41ec-bd4c-622f3bc226d4
26,6.0,6,b563cf2c-4e2e-44b2-a3ad-6cdb552cf5c7
27,55.0,7,51f1d4cd-94fc-4753-be17-ce15cc2ff401
28,99.0,8,ed9dc869-07fd-4464-bb82-68d086195993
29,54.0,9,61fb6f32-d489-4e0f-b5b0-f436a58274fb
30,24.0,0,f3b46246-34e9-426c-be12-856975d79bc5
31,6.0,1,5a88c75c-9b2a-4da8-8053-96360d155c08
32,2.0,2,8c408127-2c99-4e0f-96fc-7ea6f74cbc0c
33,90.0,3,8935f9a8-fb53-43a5-a126-2aa0768c540b


### C3. Create the countries lookup table 
Now, let's create a lookup table that maps **country_id** from the data tables to the actual country name.

The **countries** table will contain 12 rows.


In [0]:
## Drop the table if it exists
spark.sql('DROP TABLE IF EXISTS countries')

## Create the data
countries = [(0, "Italy"),
             (1, "Canada"),
             (2, "Mexico"),
             (3, "China"),
             (4, "Germany"),
             (5, "UK"),
             (6, "Japan"),
             (7, "Korea"),
             (8, "Australia"),
             (9, "France"),
             (10, "Spain"),
             (11, "USA")
            ]

columns = ["id", "name"]

## Create the Spark DataFrame and countries table
countries_df = (spark
                .createDataFrame(data = countries, schema = columns)
                .write
                .mode('overwrite')
                .saveAsTable("countries")
            )


## Display the table
display(spark.sql('SELECT * FROM countries'))

id,name
6,Japan
7,Korea
8,Australia
0,Italy
1,Canada
2,Mexico
9,France
10,Spain
11,USA
3,China


##D. Joins

Now we'll perform a query that induces shuffling by joining three tables together, writing the results to a separate table.


### D1. Disabling Broadcast Joins to Demonstrate Shuffle

These options automatically determine when a broadcast join should be used based on the size of the smaller DataFrame (or table) in the join.

- [spark.sql.autoBroadcastJoinThreshold](https://spark.apache.org/docs/3.5.3/sql-performance-tuning.html#other-configuration-options) documentation

- [spark.databricks.adaptive.autoBroadcastJoinThreshold](https://spark.apache.org/docs/3.5.3/sql-performance-tuning.html#converting-sort-merge-join-to-broadcast-join) documentation

Run the cell below to view the default values of the broadcast join and the Adaptive Query Execution(AQE) configurations. Notice that:
- The default value for a table with a broadcast join is **10,485,760 bytes**.
- If AQE is enabled, the value for the broadcast join is **31,457,280 bytes**.
- Note that AQE is enabled by default.

In [0]:
print(f'Default value of autoBroadcastJoinThreshold: {spark.conf.get("spark.sql.autoBroadcastJoinThreshold")}')
print(f'Default value of adaptive.autoBroadcastJoinThreshold: {spark.conf.get("spark.databricks.adaptive.autoBroadcastJoinThreshold")}')
print(f'View if AQE is enabled: {spark.conf.get("spark.sql.adaptive.enabled")}')

Default value of autoBroadcastJoinThreshold: 10485760b
Default value of adaptive.autoBroadcastJoinThreshold: 31457280b
View if AQE is enabled: true


In this cell, we're explicitly turning off broadcast joins in order to demonstrate a shuffle and show you how to investigate and improve the query performance.

In [0]:
## Disabling the automatic broadcast join entirely. That is, Spark will never broadcast any dataset for joins, regardless of its size.
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

# Disabling the broadcast join feature under AQE, meaning that even when using adaptive query execution, Spark will not attempt to broadcast any smaller side of a join.
spark.conf.set("spark.databricks.adaptive.autoBroadcastJoinThreshold", -1)


## View the new values
print(f'Current value of autoBroadcastJoinThreshold: {spark.conf.get("spark.sql.autoBroadcastJoinThreshold")}')
print(f'Current value of adaptive.autoBroadcastJoinThreshold: {spark.conf.get("spark.databricks.adaptive.autoBroadcastJoinThreshold")}')

Current value of autoBroadcastJoinThreshold: -1
Current value of adaptive.autoBroadcastJoinThreshold: -1


Perform the join of **transactions**, **stores**, and **countries** to create a table named **transact_countries** without using broadcast joins. Notice that this query takes about ~1 minute to execute.


In [0]:
joined_df_no_broadcast = spark.sql("""
    SELECT 
        transactions.id,
        amount,
        countries.name as country_name,
        employees,
        stores.name as store_name
    FROM
        transactions
    LEFT JOIN
        stores
        ON
            transactions.store_id = stores.id
    LEFT JOIN
        countries
        ON
            transactions.country_id = countries.id
""")

## Create a table with the joined data
(joined_df_no_broadcast
 .write
 .mode('overwrite')
 .saveAsTable('transact_countries')
)

Open the Spark UI to the Stages page and notice that there were two large **1.4GB** shuffles of data. 

A shuffle refers to the process of redistributing data across different nodes/partitions. This is necessary for operations like joins, where data from two or more datasets needs to be combined based on a key, but the relevant rows might not be on the same partition.


To view how the query DAG, complete the following steps:

1. In the cell above, expand **Spark Jobs**.

2. In the second job, right-click on **View** and select *Open in a New Tab*.

    **NOTE:** In the Vocareum lab environment, if you click **View** without opening it in a new tab, the pop-up window will display an error.

3. Click on the **Stages** link in the top navigation bar. Notice that in the stages of this specific query, there were very large **Shuffle Read** and **Shuffle Write** operations for the query.

    ![1.3-non_broadcast_join_stages.png](./Includes/images/1.3-non_broadcast_join_stages.png)

4. Next, in the **Completed Stages** table select the link (**mapPartitionsInternal at...**) in the **Description** column for the row that has the large **Shuffle Read** and **Shuffle Write** the **Jobs** link in the top navigation bar. For example, in the image above it would be row **151**.

5. Then, select number link in the table at the top in **Associated Jobs Ids** -> then the query number in **Associated SQL Query**. Here, you should see the entire query plan (DAG).

    ![1.3-non_broadcastjoin_dag.png](./Includes/images/1.3-non_broadcastjoin_dag.png)

<br></br>
6. In the query plan, complete the following steps:
<br></br>
##### 6a. Shuffle Join with transactions and stores

![1.3-shuffle_join_transactions_stores.png](./Includes/images/1.3-shuffle_join_transactions_stores.png)

- Find **PhotonScan parquet dbacademy.your-schema-name.transactions (1)**.
  
- Above that table, expand **PhotonShuffleExchangeSink (2)**.

- Find the metric **num bytes written total (min, med, max)**. Notice that the shuffle uses around 1.4GB for the first join with the **stores** table.
<br></br>
##### 6b. Shuffle Join with transactions and stores

![1.3-shuffle_join_results_countries.png](./Includes/images/1.3-shuffle_join_results_countries.png)

- Find **PhotonShuffleExchangeSource** on the left side above the **AQEShuffleRead** for the result of the previous join.

- Expand **PhotonShuffleExchangeSource**.

- Find the metric **num bytes written total (min, med, max)**. Notice that another shuffle was performed with around 1.4GB for the join of the results of the first join to the join with the **countries** table.


###D2. Enabling Broadcast Join

Broadcast join avoids the shuffle. In the above cells, we explicitly turned off broadcast joins, but now we'll return the configuration to the default so that broadcast join is enabled to compare the queries. 

This will work in this case because at least one of the tables in each join is relatively small and below the thresholds:
- < 10MB for a broadcast join without AQE
- < 30MB for a broadcast join with AQE


In [0]:
# Set the default configurations for broadcast joins
spark.conf.unset("spark.sql.autoBroadcastJoinThreshold")
spark.conf.unset("spark.databricks.adaptive.autoBroadcastJoinThreshold")

## View the default values
print(f'Current value of autoBroadcastJoinThreshold: {spark.conf.get("spark.sql.autoBroadcastJoinThreshold")}')
print(f'Current value of adaptive.autoBroadcastJoinThreshold: {spark.conf.get("spark.databricks.adaptive.autoBroadcastJoinThreshold")}')

Run the query to perform the same join as before. Notice that this query executes in about half the time of the previous join.

In [0]:
joined_df = spark.sql("""
    SELECT 
        transactions.id,
        amount,
        countries.name as country_name,
        employees,
        stores.name as store_name
    FROM
        transactions
    LEFT JOIN
        stores
        ON
            transactions.store_id = stores.id
    LEFT JOIN
        countries
        ON
            transactions.country_id = countries.id
""")

(joined_df
 .write
 .mode('overwrite')
 .saveAsTable('transact_countries')
)


This is an improvement. Referring back to the Spark UI **Stages** tab, note that there are no large shuffle reads anymore. Only the small tables were shuffled, but the large table was not, so we avoided moving 1.4GB twice.


View the stages of this query. Notice that the large shuffles have been avoided, improving the query efficiency.
![1.3-broadcast_join_stages.png](./Includes/images/1.3-broadcast_join_stages.png)

<br></br>
You can also view the query plan and see that the large tables were not shuffled.

![1.3-broadcast_join.png](./Includes/images/1.3-broadcast_join.png)

<br></br>
Broadcast joins can be significantly faster than shuffle joins if one of the tables is very large and the other is small. Unfortunately, broadcast joins only work if at least one of the tables is less than 100MB in size. In the case of joining larger tables, if we want to avoid the shuffle, we may need to reconsider our schema to avoid having to do the join in the first place.


## E. Aggregations

Aggregations also use a shuffle, but they're often much less expensive. The following cell executes a query that demonstrates this.

In [0]:
%sql
SELECT 
  country_id, 
  COUNT(*) AS count,
  AVG(amount) AS avg_amount
FROM transactions
GROUP BY country_id
ORDER BY count DESC

That was fast! There are a lot of things going on here. One of the main points is that we're only shuffling the counts and sums necessary to compute the counts and averages that were requested. This only results in shuffling a few KB. Use the Spark UI once again to verify this.

![1.3_aggregations-ui.png](./Includes/images/1.3-aggregations_ui.png)

So the shuffle is cheap compared to the shuffle joins, which need to shuffle all of the data. It also helps that our output is basically 0 in this case.


&copy; 2025 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the 
<a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/><a href="https://databricks.com/privacy-policy">Privacy Policy</a> | 
<a href="https://databricks.com/terms-of-use">Terms of Use</a> | 
<a href="https://help.databricks.com/">Support</a>